In [1]:
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
interactions_path = "./data/RAW_interactions.csv"
recipes_path = "./data/RAW_recipes.csv"

### Loading Data

In [4]:
interactions_raw = pd.read_csv(interactions_path)
recipes_raw = pd.read_csv(recipes_path)

In [5]:
interactions_raw.head(1)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...


In [6]:
recipes_raw.head(1)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7


In [7]:
interactions_raw.groupby('recipe_id').head(5)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


In [8]:
interactions_raw.groupby(['recipe_id', 'rating'])['user_id'].agg('count').reset_index()

,recipe_id,rating,user_id
0,38,4,3
1,38,5,1
2,39,3,1
3,40,1,1
4,40,4,2
...,...,...,...
367682,537459,5,1
367683,537485,5,1
367684,537543,0,1
367685,537671,0,1


In [9]:
K = 6
def count_ratings(x):
    ret = [0] * K
    ratings = list(x.rating)
    counts = list(x.user_id)
    for rating, count in zip(ratings, counts):
        ret[K - rating - 1] = count
    return ret

In [10]:
recipes_with_rating_counts = interactions_raw.groupby(['recipe_id', 'rating'])['user_id'].agg('count').reset_index().groupby('recipe_id')[['rating', 'user_id']].apply(count_ratings).reset_index(name='ratings')

In [11]:
import math
def starsort(ns):
    """
    http://www.evanmiller.org/ranking-items-with-star-ratings.html
    """
    N = sum(ns)
    K = len(ns)
    s = list(range(K,0,-1))
    s2 = [sk**2 for sk in s]
    z = 1.65
    def f(s, ns):
        N = sum(ns)
        K = len(ns)
        return sum(sk*(nk+1) for sk, nk in zip(s,ns)) / (N+K)
    fsns = f(s, ns)
    return fsns - z*math.sqrt((f(s2, ns)- fsns**2)/(N+K+1))

In [12]:
recipes_with_rating_counts['popularity'] = recipes_with_rating_counts['ratings'].apply(starsort)

In [13]:
recipes_with_rating_counts.head()

,recipe_id,ratings,popularity
0,38,"[1, 3, 0, 0, 0, 0]",3.404010
1,39,"[0, 0, 1, 0, 0, 0]",2.643421
2,40,"[6, 2, 0, 0, 1, 0]",3.896731
3,41,"[1, 1, 0, 0, 0, 0]",3.047372
4,43,"[0, 0, 0, 0, 1, 0]",2.313839


In [14]:
recipes_with_rating_counts.sort_values('popularity', ascending=False).head()

,recipe_id,ratings,popularity
224586,486261,"[215, 2, 0, 0, 0, 0]",5.868667
23999,42976,"[112, 5, 0, 0, 0, 1]",5.679877
53087,92072,"[121, 11, 1, 0, 0, 1]",5.654607
13495,25723,"[161, 23, 1, 2, 0, 1]",5.647498
42195,73488,"[117, 16, 1, 1, 0, 0]",5.643824


In [15]:
# Get recommended recipes for a given user id - call popularity model with recipes_with_rating_counts and a given user id
from models import PopularityModel
popularity_data = recipes_with_rating_counts.sort_values('popularity', ascending=False)
model = PopularityModel.PopularityModel(popularity_data)

In [16]:
model.recommend(38094, interactions_raw)

[486261, 42976, 92072, 25723, 73488, 69173, 100526, 106251, 8739, 31128]

#### Get the top 10 most popular recipes

In [17]:
n = 10
top_n_recipes = list(recipes_with_rating_counts.sort_values('popularity', ascending=False).head(n).recipe_id)

In [18]:
top_n_recipes

[486261, 42976, 92072, 25723, 73488, 69173, 100526, 106251, 8739, 31128]

In [19]:
recipes_raw.set_index('id').loc[top_n_recipes].reset_index(inplace=False)

,id,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,486261,mexican stack up rsc,75,226863,2012-09-01,"['time-to-make', 'course', 'main-ingredient', ...","[793.0, 83.0, 10.0, 34.0, 87.0, 131.0, 11.0]",28,"['preheat broiler to high broil', 'set the rac...","ready, set, cook! reynolds wrap contest entry....","['poblano pepper', 'reynolds wrap foil', 'oliv...",22
1,42976,brown sugar bundt cake,90,35047,2002-10-14,"['weeknight', 'time-to-make', 'course', 'cuisi...","[637.4, 57.0, 211.0, 21.0, 11.0, 25.0, 24.0]",9,['grease a 10-inch bundt pan well and sprinkle...,my mom passed this recipe along to me. it is e...,"['yellow cake mix', 'instant vanilla flavor pu...",7
2,92072,monterey chicken rolls,75,76951,2004-05-28,"['time-to-make', 'course', 'main-ingredient', ...","[455.6, 42.0, 4.0, 30.0, 78.0, 55.0, 3.0]",6,"['pound chicken breasts flat', 'slice monterey...",butter...garlic...cheese? chicken rolled in al...,"['boneless skinless chicken breasts', 'montere...",7
3,25723,tiny cinnamon rolls,42,37779,2002-04-20,"['60-minutes-or-less', 'time-to-make', 'course...","[44.2, 1.0, 11.0, 2.0, 2.0, 0.0, 2.0]",10,"['unroll dough , and separate into 4 rectangle...",southern living. my kids adore these tiny litt...,"['refrigerated crescent dinner rolls', 'sugar'...",6
4,73488,individual cream cheese danish,40,101823,2003-10-16,"['60-minutes-or-less', 'time-to-make', 'course...","[87.6, 4.0, 24.0, 3.0, 3.0, 7.0, 4.0]",8,"['preheat oven to 350 degrees f', 'slice cresc...",small danishes that make a great addition to a...,"['crescent roll dough', 'cream cheese', 'white...",9
5,69173,kittencal s italian melt in your mouth meatballs,50,89831,2003-08-20,"['60-minutes-or-less', 'time-to-make', 'course...","[1312.6, 129.0, 8.0, 108.0, 214.0, 174.0, 8.0]",5,['mix all ingredients together in a large bowl...,cooking the meatballs in simmering pasta sauce...,"['ground beef', 'egg', 'parmesan cheese', 'bre...",10
6,100526,sweet creamy coleslaw,15,1535,2004-09-24,"['15-minutes-or-less', 'time-to-make', 'course...","[216.1, 20.0, 50.0, 13.0, 4.0, 9.0, 8.0]",6,"['shred cabbage and carrots into a bowl', 'mix...",i never liked coleslaw until i tried my husban...,"['green cabbage', 'carrots', 'mayonnaise', 'vi...",5
7,106251,roasted cauliflower 16 roasted cloves of garlic,40,58104,2004-12-16,"['60-minutes-or-less', 'time-to-make', 'course...","[126.9, 14.0, 11.0, 17.0, 6.0, 6.0, 3.0]",4,"['mix oil , rosemary , salt , pepper and garli...","i find it totally amazing, that children and m...","['cauliflower', 'garlic cloves', 'fresh rosema...",6
8,8739,wholly guacamole,20,7802,2001-04-09,"['30-minutes-or-less', 'time-to-make', 'course...","[500.7, 68.0, 14.0, 0.0, 13.0, 32.0, 10.0]",13,"['cut avocados in half , remove pit , and spoo...",this is a true tex-mex recipe concocted by a d...,"['avocados', 'garlic cloves', 'lime', 'tomatoe...",8
9,31128,yummy crunchy apple pie,70,29450,2002-06-13,"['weeknight', 'time-to-make', 'course', 'main-...","[360.6, 22.0, 161.0, 6.0, 5.0, 38.0, 19.0]",15,['use foil on bottom of oven rather than a coo...,great pie!,"['pastry for single-crust pie', 'sugar', 'flou...",12


### Content Based Filtering Recommendation System

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import ast

In [21]:
recipes = recipes_raw.copy()

In [22]:
#TODO: do text pre-processing
recipes.description = recipes.description.fillna('')
recipes.name = recipes.name.fillna('')

In [23]:
recipes.ingredients = recipes.ingredients.apply(lambda x: ' '.join(ast.literal_eval(x)))

In [24]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(recipes.name + " " + recipes.description + " " + recipes.ingredients)

In [25]:
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,winter squash mexican seasoning mixed spice ho...,7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,prepared pizza crust sausage patty eggs milk s...,6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,ground beef yellow onions diced tomatoes tomat...,13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...",spreadable cheese with garlic and herbs new po...,11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,tomato juice apple cider vinegar sugar salt pe...,8


In [46]:
from models import ContentBasedModel
model = ContentBasedModel.ContentBasedModel(recipes, interactions_raw)

In [50]:
indices = model.recommend(38094, pd.DataFrame.sparse.from_spmatrix(X))

In [51]:
recipes.iloc[indices]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
3298,all american county fair prize winning chili,57204,360,43478,2003-03-26,"['time-to-make', 'course', 'main-ingredient', ...","[154.8, 5.0, 23.0, 11.0, 25.0, 6.0, 6.0]",12,['put all ingredients in a pot and cook for 5 ...,after my five roommates and i have this for di...,tomatoes tomato sauce tomato paste water onion...,27
172136,red lentil carrot and roasted red pepper soup,168228,30,67009,2006-05-16,"['30-minutes-or-less', 'time-to-make', 'course...","[322.4, 11.0, 26.0, 31.0, 39.0, 20.0, 15.0]",7,['put the first 8 ingredients into a stock pot...,i made this soup by experimenting with ingredi...,dried red lentils onion red pepper celery rib ...,17
196021,spicy honey glazed chicken,271659,20,666723,2007-12-13,"['30-minutes-or-less', 'time-to-make', 'course...","[505.9, 44.0, 105.0, 30.0, 66.0, 41.0, 9.0]",11,"['in a large bowl , mix together first 7 ingre...",this is a simple worknight dish that will tota...,garlic powder chili powder salt pepper ground ...,10
173440,ribollita minestrone with bread,308540,60,37722,2008-06-10,"['60-minutes-or-less', 'time-to-make', 'course...","[380.0, 11.0, 24.0, 25.0, 32.0, 7.0, 21.0]",17,"['pre-heat the broiler', 'place bread slices o...","i love this light, easy ribollita (a minestron...",french bread garlic clove parmesan cheese oliv...,17
73117,double dredge fried chicken,351571,30,726484,2009-01-23,"['30-minutes-or-less', 'time-to-make', 'course...","[512.5, 7.0, 1.0, 135.0, 80.0, 6.0, 24.0]",16,"['rinse chicken', 'place chicken and salt in a...",this is my family's favorite fried chicken! i ...,boneless skinless chicken breasts salt all-pur...,7
170319,raisin orange salsa,176700,80,302961,2006-07-10,"['time-to-make', 'course', 'main-ingredient', ...","[106.1, 0.0, 75.0, 0.0, 3.0, 0.0, 9.0]",1,"['combine all ingredients , cover and refriger...","this is an excellent salsa to serve with pork,...",raisins red bell pepper yellow bell pepper gre...,14
67640,cuban black beans,76782,645,97895,2003-11-17,"['weeknight', 'time-to-make', 'course', 'main-...","[130.3, 6.0, 9.0, 3.0, 11.0, 3.0, 6.0]",13,"['place 1 quart stock in pot set on high', 'sa...","i got my first recipe for ""frijoles negro cuba...",olive oil onions celery green bell pepper red ...,23
46000,chicken tenders with spicy rice and red peppers,39752,50,47559,2002-09-09,"['60-minutes-or-less', 'time-to-make', 'course...","[500.5, 18.0, 10.0, 46.0, 68.0, 10.0, 20.0]",20,"['heat oven to 300 degrees', 'mix cilantro , c...",a flavorful chicken tender recipe.,"cilantro ground cumin hot paprika salt lime, j...",13
141544,my chicken parmigiana with homemade pasta sauce,254738,270,166642,2007-09-23,"['course', 'main-ingredient', 'cuisine', 'prep...","[1049.2, 79.0, 207.0, 168.0, 129.0, 82.0, 29.0]",18,"['pasta sauce:', 'in dutch oven or large sauce...",this is my version of chicken parmigiana. the ...,olive oil onion garlic cloves tomato paste wat...,22
227965,wolfgang puck s no cook gazpacho,27022,15,34146,2002-05-02,"['celebrity', '15-minutes-or-less', 'time-to-m...","[447.0, 59.0, 49.0, 83.0, 10.0, 27.0, 9.0]",14,"['---prepare gazpacho:', 'in a large bowl , co...",this recipe for wolfgang puck's no cook gazpac...,roma tomatoes red bell pepper english cucumber...,26


In [53]:
recipes.iloc[list(model._get_user_reviews(38094))]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
40893,cheesy tortellini in a creamy sauce,384231,35,37449,2009-08-05,"['60-minutes-or-less', 'time-to-make', 'course...","[660.2, 54.0, 19.0, 42.0, 53.0, 88.0, 20.0]",16,['place the olive oil and butter in a large sk...,"i created this for the ready set cook contest,...",napa cabbage olive oil butter garlic cloves gr...,16
16954,banana salsa,122374,10,153188,2005-05-16,"['15-minutes-or-less', 'time-to-make', 'course...","[89.9, 0.0, 55.0, 4.0, 2.0, 0.0, 7.0]",1,['mix all ingredients together and serve'],"this is an amazing addition to any pork, fish ...",bananas yellow bell pepper jalapeno chiles fre...,8
40753,cheesy sausage meatballs,55636,25,26278,2003-03-05,"['30-minutes-or-less', 'time-to-make', 'course...","[385.5, 36.0, 4.0, 30.0, 46.0, 38.0, 6.0]",5,"['mix ingredients well', 'make into 20 meatbal...",a different type of meatball that works well i...,bulk sausage plain breadcrumbs milk cornmeal p...,9
34513,can t leave alone bars,178800,40,173314,2006-07-24,"['60-minutes-or-less', 'time-to-make', 'course...","[687.0, 51.0, 296.0, 23.0, 18.0, 63.0, 30.0]",7,"['mix cake mix , oil , and eggs in bowl', 'pre...",delicious make ahead dessert!my friend chris g...,white cake mix eggs sweetened condensed milk b...,6
69545,danish cherries,369282,25,37449,2009-05-04,"['celebrity', 'danish', '30-minutes-or-less', ...","[149.1, 7.0, 77.0, 0.0, 5.0, 1.0, 8.0]",7,['place cherries in a heavy medium-sized sauce...,a sweet scandinavian recipe adapted from moose...,cherries cornstarch sugar lemon juice lemon ze...,11
49064,choc chip muesli bars,246450,10,315565,2007-08-14,"['15-minutes-or-less', 'time-to-make', 'course...","[196.0, 11.0, 59.0, 2.0, 4.0, 24.0, 11.0]",6,"['combine first 6 ingredients', 'place butter ...",quick and yummy - adapted from a nestle brochure,rice bubbles rolled oats coconut dried fruit c...,9
80044,elegant chicken zucchini skewers,377275,60,485109,2009-06-15,"['lactose', '60-minutes-or-less', 'time-to-mak...","[341.3, 31.0, 11.0, 4.0, 64.0, 24.0, 2.0]",9,['thinly slice the chicken lengthwise so you e...,this is an elegant way to serve meat and veget...,chicken breasts zucchini cayenne pepper ground...,8
30565,brownies with white chocolate chips and raspbe...,374738,50,1278433,2009-05-28,"['60-minutes-or-less', 'time-to-make', 'course...","[254.3, 16.0, 104.0, 10.0, 9.0, 31.0, 13.0]",16,"['preheat the oven to 350 degrees', 'whisk the...",raspberries and chocolate are a naturally invi...,flour sugar unsweetened cocoa baking powder sa...,10
29493,broccoli salad made easy,102765,195,71069,2004-10-27,"['time-to-make', 'course', 'main-ingredient', ...","[430.8, 56.0, 47.0, 27.0, 16.0, 46.0, 6.0]",2,"['mix everything together', 'chill several hou...","this is a delicious ""mix and chill""! the most ...",broccoli floret coleslaw dressing bacon white ...,6
34509,can t get any easier tuna noodle casserole,11535,11,9441,2001-09-08,"['15-minutes-or-less', 'time-to-make', 'course...","[178.1, 6.0, 7.0, 18.0, 24.0, 5.0, 7.0]",4,['cook noodles according to package directions...,"i know, this isn't technically a ""casserole,"" ...",egg noodles light chunk tuna in water condense...,5
